<a href="https://colab.research.google.com/github/gnicoleta/BDA/blob/main/FCM_IMPL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
#open dataset file from drive
from google.colab import drive

drive.mount('/content/drive')


dataset_size = 150
data_file_path = "/content/drive/MyDrive/BDA/Dataset1/" + f'Iris-{str(dataset_size)}.txt'
open(data_file_path, 'r')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<_io.TextIOWrapper name='/content/drive/MyDrive/BDA/Dataset1/Iris-150.txt' mode='r' encoding='UTF-8'>

In [33]:
#load data
import numpy as np
import pandas as pd

np.random.seed(50)

data_frame = pd.read_csv(data_file_path, header=None)
iris_data = data_frame.values

#fara label -> ai nevoie doar de nr ca sa poti calcula distanta
data = iris_data[:, :iris_data.shape[1]-1]

data.shape


(150, 4)

In [34]:
#init centroids
import numpy as np

k = 3
random_centroids = data_frame.sample(n=k).values[:, :-1] #dont need the last value, the label
print("Random: ", random_centroids)

print(data[0]) #first
print(data[-1]) #last
print(data[round(len(data)/2)+1]) #middle

predefined_centroids = np.empty((0, 4))
predefined_centroids = np.append(predefined_centroids, np.array([data[0]]), axis=0)
predefined_centroids = np.append(predefined_centroids, np.array([data[-1]]), axis=0)
predefined_centroids = np.append(predefined_centroids, np.array([data[round(len(data)/2)]]), axis=0)
print("\nPredefined: ", predefined_centroids)

Random:  [[5.6 3.0 4.1 1.3]
 [6.3 2.5 4.9 1.5]
 [5.4 3.4 1.7 0.2]]
[5.1 3.5 1.4 0.2]
[5.9 3.0 5.1 1.8]
[6.8 2.8 4.8 1.4]

Predefined:  [[5.1 3.5 1.4 0.2]
 [5.9 3.0 5.1 1.8]
 [6.6 3.0 4.4 1.4]]


In [35]:
import numpy as np
from numpy.linalg import norm

def euclidian_distance(a, b): 
  return norm(np.array(a)-np.array(b)) #ultima valoare este un label, nu poti calcual distanta de string, doar de nr


In [234]:
import math
m = 3

def compute_membership_degree(data_point, centroids):

  p = float(2/(m-1))
  distances_to_centroids = [] #lista care va contine cele k distante de la punct la fiecare centroid

  suma_0 = 0
  for j in range(k):
    suma_0 += math.pow((1/euclidian_distance(data_point, centroids[j])), p)
  membership_degrees = []
  for centroid in centroids:
    aux1 = math.pow((1/euclidian_distance(data_point, centroid)),p)
    membership_degrees.append(aux1/suma_0)

  closest_centroid_index = np.argmax(membership_degrees)
  return closest_centroid_index, membership_degrees

def compute_clusters(data, centroids, k):
  clusters = [[] for _ in range(k)]
  
  membership_matrix = []
  for data_point_index, data_point in enumerate(data):
    (centroid_index, membership_degree) = compute_membership_degree(data_point, centroids)
    membership_matrix.append(membership_degree)
    clusters[centroid_index].append(data_point_index)
  return clusters, membership_matrix

def compute_new_centroids_FCM(k, data_points, membership_matrix):
  new_centroids = []
  for j in range(k):
    u = sum(np.power(membership_matrix[:][j],m))
    cx_sum = 0
    for i in range(150):
      #print("DODO: ", np.power(membership_matrix[i][j],m), np.multiply(np.power(membership_matrix[i][j],m),data_points[i,:]))
      c = np.multiply(np.power(membership_matrix[i][j],m),data_points[i,:])
      if(not math.isnan(c[j])):
        cx_sum += c[j]
        #print("For: ",j, c[j], cx_sum)
    new_centroid_value = cx_sum/u
    print("WTF: ", cx_sum, u, new_centroid_value)
    new_centroids.append(new_centroid_value)
  print("LAST: ", new_centroids)
  return new_centroids

#verifica daca distanta dintre centroizi s-a schimbat
def compute_sse(k, centroids_old, centroids):
  distances = [euclidian_distance(centroids_old[i], centroids[i]) for i in range(k)]
  return sum(distances)

(clst, membmtrx) = compute_clusters(data, random_centroids, 3)
compute_new_centroids_FCM(3, data, membmtrx)

WTF:  53.90126607146747 0.5105283735066718 105.57937397530965
WTF:  36.29206332039362 0.3619766945582471 100.2607733204595
WTF:  29.84232667670495 0.328732531550315 90.77996186129621
LAST:  [105.57937397530965, 100.2607733204595, 90.77996186129621]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: divide by zero encountered in double_scalars
  


[105.57937397530965, 100.2607733204595, 90.77996186129621]

In [235]:
import numpy as np
import time

def FCM(max_iterations, k, error_margin, data=data, centroids=predefined_centroids):
  print(centroids)
  
  clusters=[[] for _ in range (k)]

  for i in range(max_iterations):
    print("\n\n")
    start = time.time()
    (clusters, membership_matrix)= compute_clusters(data, centroids, k)
    #tine minte vechii centroizi ca sa vezi convergenta
    centroids = compute_new_centroids_FCM(k, data, membership_matrix)

    print("erwrew",membership_matrix)
    print("dasa", membership_matrix[i][0])
    print("dasa", membership_matrix[i][1])
    if(abs(membership_matrix[i][0]-membership_matrix[i][1])<=error_margin):
      stop = time.time()
      duration = stop-start
      print("Iteration ", i+1, " algoritm has converged, variance: ", ", duration: ", duration)
      break
      
    stop = time.time()
    duration = stop-start
    print("Iteration ", i+1, ", variance: ", ", duration: ", duration)
  
    for _ in range (k):
        print("\tCluster ", _, ": ", len(clusters[_]))

  for _ in range (k):
        print("\n Cluster ", _, ": ", len(clusters[_]))
  return (clusters, centroids)
      

In [236]:
print(random_centroids)
(resulted_clusters, optim_centroids) = FCM(7, 3, 0, data, random_centroids)

[[5.6 3.0 4.1 1.3]
 [6.3 2.5 4.9 1.5]
 [5.4 3.4 1.7 0.2]]
[[5.6 3.0 4.1 1.3]
 [6.3 2.5 4.9 1.5]
 [5.4 3.4 1.7 0.2]]



WTF:  53.90126607146747 0.5105283735066718 105.57937397530965
WTF:  36.29206332039362 0.3619766945582471 100.2607733204595
WTF:  29.84232667670495 0.328732531550315 90.77996186129621
LAST:  [105.57937397530965, 100.2607733204595, 90.77996186129621]
erwrew [[0.1159592200873995, 0.0859546926503391, 0.7980860872622614], [0.16704594253927532, 0.12463198551841191, 0.7083220719423128], [0.1806446226365912, 0.13510020321956145, 0.6842551741438474], [0.1940713389368955, 0.14317183735233116, 0.6627568237107734], [0.13552938292864056, 0.1002831293945808, 0.7641874876767787], [0.14747873737556655, 0.10653236422327703, 0.7459888984011565], [0.18829842488484907, 0.13880745781349668, 0.6728941173016543], [0.12121851162566306, 0.08899846440290769, 0.7897830239714292], [0.2234883265177652, 0.1675163120726605, 0.6089953614095742], [0.15469792311877512, 0.11464800999183253, 0.7306540668

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: divide by zero encountered in double_scalars
  
